<a href="https://colab.research.google.com/github/dsandeep0138/HackerEarth-Challenges/blob/master/flower_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q kaggle

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"dsandeep97","key":"91ed93d9f57d24d723707d120074f706"}'}

In [0]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [0]:
 !chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d rednivrug/flower-recognition-he

 99% 897M/905M [00:07<00:00, 141MB/s]
100% 905M/905M [00:07<00:00, 129MB/s]


In [0]:
!unzip flower-recognition-he.zip

In [8]:
import gc, cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau

from keras import applications
from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [0]:
traindf = pd.read_csv('data/train.csv', dtype=str)
testdf = pd.read_csv('data/test.csv', dtype=str)

In [0]:
traindf['image_id'] = traindf['image_id'] + ".jpg"
testdf['image_id'] = testdf['image_id'] + ".jpg"

In [11]:
datagen = ImageDataGenerator(rescale=1./255.,
                             rotation_range=10,
                             width_shift_range=0.1, 
                             height_shift_range=0.1)

train_generator = datagen.flow_from_dataframe(
dataframe=traindf,
directory="./data/train/",
x_col="image_id",
y_col="category",
subset="training",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(224, 224))

Found 18540 validated image filenames belonging to 102 classes.


In [12]:
test_datagen=ImageDataGenerator(rescale=1./255.)

test_generator=test_datagen.flow_from_dataframe(
dataframe=testdf,
directory="./data/test/",
x_col="image_id",
y_col=None,
batch_size=1,
seed=42,
shuffle=False,
class_mode=None,
target_size=(224, 224))

Found 2009 validated image filenames.


In [13]:
vgg_model = applications.VGG19(weights = "imagenet", include_top=True, input_shape = (224, 224, 3))

model = Sequential()
for layer in vgg_model.layers[:-1]:
    model.add(layer)

for layer in model.layers[:]:
    layer.trainable=False

model.add(Dense(102, activation='softmax'))

model.summary()





574717952/574710816 [==============================] - 6s 0us/step


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_______________________________________________

In [14]:
model.compile(loss="categorical_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='acc',
                                factor=0.5,
                                patience=3,
                                min_lr=0.00001,
                                verbose=1)

In [15]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    epochs=30,
                    verbose=1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/30
579/579 [==============================] - 282s 488ms/step - loss: 3.1377 - acc: 0.2977
Epoch 2/30
579/579 [==============================] - 280s 484ms/step - loss: 2.0179 - acc: 0.4939
Epoch 3/30
579/579 [==============================] - 279s 482ms/step - loss: 1.6766 - acc: 0.5639
Epoch 4/30
579/579 [==============================] - 281s 485ms/step - loss: 1.4942 - acc: 0.6041
Epoch 5/30
579/579 [==============================] - 282s 487ms/step - loss: 1.3685 - acc: 0.6323
Epoch 6/30
579/579 [==============================] - 281s 485ms/step - loss: 1.2952 - acc: 0.6520
Epoch 7/30
579/579 [==============================] - 279s 481ms/step - loss: 1.1911 - acc: 0.6706
Epoch 8/30
579/579 [==============================] - 278s 480ms/step - loss: 1.1294 - acc: 0.6860
Epoch 9/30
579/579 [==============================] - 276s 477ms/step - loss: 1.0991 - acc: 0.6939
Epoch 10/30
579

In [16]:
test_generator.reset()
pred = model.predict_generator(test_generator,
                               steps=STEP_SIZE_TEST,
                               verbose=1)

2009/2009 [==============================] - 65s 32ms/step


In [0]:
predicted_class_indices=np.argmax(pred, axis=1)

In [0]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [0]:
filenames=test_generator.filenames
filenames=[f.split('.')[0] for f in filenames]

results=pd.DataFrame({"image_id":filenames,
                      "category":predictions})
results = results.sort_values(by = ['image_id'], ascending = [True])
results.to_csv("results.csv", index=False)